In [99]:
import numpy as np
import scipy
from scipy.stats import norm
import math
import random
import operator

In [126]:
class knn:
    """kNearest Neighbour classification implementation"""
    
    def __init__(self,k):
        self.k = k
        
    def euclideanDistance(self,data1,data2,n):
        distance = 0
        for i in range(0,n):
            distance+=np.square(data1[i]-data2[i])
        return np.sqrt(distance)
    
    def manhattanDistance(self,data1,data2,n):
        distance = 0
        for i in range(n):
            distance+=abs(data1[i]-data2[i])
        return distance
    
    def hammingDistance(self,data1,data2,n):
        distance = 0
        for i in range(n):
            if data1[i]==data2[i]:
                distance+=1
        return distance
    
    ##using euclidean distance for now
    def getNeighbours(self,dataset,data):
        distances = []
        length = len(data)
#         print("length:"+str(length))
        for i in range(0,len(dataset)):
            ##change the below line for different distances
#             print("i in neighbour: "+str(i))
            dist = self.euclideanDistance(data,dataset[i],length)
            distances.append((dataset[i],dist))
        distances.sort(key=operator.itemgetter(1))
        neighbours = []
        for i in range(self.k):
            neighbours.append(distances[i][0])
        return neighbours
    
    def getClass(self,neighbours):
        classOccurrence = {}
        for i in range(len(neighbours)):
            t = neighbours[i][-1]
            if t in classOccurrence:
                classOccurrence[t]+=1
            else:
                classOccurrence[t] = 1
        sortedOccurrence = sorted(classOccurrence.items(),key=operator.itemgetter(1),reverse=True)
        return sortedOccurrence[0][0]

In [101]:
import os
import sys

In [102]:
scriptpath = "./fashion-mnist/utils"

# Add the directory containing your module to the Python path (wants absolute paths)
sys.path.append(os.path.abspath(scriptpath))

In [103]:
import mnist_reader

In [139]:
X_train, Y_train = mnist_reader.load_mnist('./fashion-mnist/data/fashion', kind='train')
X_test, Y_test = mnist_reader.load_mnist('./fashion-mnist/data/fashion', kind='t10k')

In [140]:
print(X_test.shape)
print(Y_test.shape)
print(X_train.shape)
print(Y_train.shape)

(10000, 784)
(10000,)
(60000, 784)
(60000,)


In [141]:
# print(Y_test[0])
b = np.ones((X_train.shape[0],1))
x = np.hstack((X_train,b))
for i in range(X_train.shape[0]):
    x[i][X_train.shape[1]]=Y_train[i]

In [142]:
class PCA:
    def __init__(self,m):
        self.m = m
        
    def dimensionReduction(self,data):
        # data is of the form n X d , with d dimensions
        self.d = data.shape[1]
        self.n = data.shape[0]
        #data -= np.mean(data,axis=0)
        S = np.cov(data.T)
#         print(S.shape)
        w,v = np.linalg.eig(S)
        #w is the array of eigen values and v's are the corresponding eigen vectors
        # for w[i] -> v[:,i] is the corresponding eigen vectors
        tempList = []
        for i in range(self.d):
            tempList.append((w[i],i))
        tempList.sort(reverse=True)
        tempVecList = []
        for i in range(self.m):
            tempVecList.append(v[:,tempList[i][1]])
        finalVecMat = np.array(tempVecList) # m X d
        reducedMat = np.matmul(finalVecMat,data.T) # m X n
        return reducedMat.T # n X m , dimension reduced from d -> m
            

In [150]:
k=2
while k!=-1:
    pcaObj = PCA(k)
    reducedTrain = pcaObj.dimensionReduction(X_train)
    reducedTest = pcaObj.dimensionReduction(X_test)
    
    b = np.ones((reducedTrain.shape[0],1))
    x = np.hstack((reducedTrain,b))
    for i in range(reducedTrain.shape[0]):
        x[i][reducedTrain.shape[1]]=Y_train[i]
    
    j=2
    while j!=-1:
        obj = knn(j)
        count=0
        limit =1000
        if k>200:
            limit = 500
        for i in range(limit+1):
            nei = obj.getNeighbours(x,reducedTest[i])
            ans = obj.getClass(nei)
#             print("our ans: "+str(ans)+" ,original ans:"+str(Y_test[i])+" for i:"+str(i))
            if ans != Y_test[i]:
                count+=1
#             print(i)
#             print(count)
            if i%10==0:
                print("pca(m): "+str(k)+",k: "+str(j)+",count: "+str(count)+",i:"+str(i))
        print("pca(m): "+str(k)+",k: "+str(j)+",count: "+str(count))
        if j==2:
            j=100
        elif j==100:
            j=1000
        else:
            j=-1
    if k==512:
        k=700
    elif k==700:
        k=-1
    elif k==2:
        k=50
    elif k==50:
        k=150
    elif k==150:
        k=512

pca(m): 2,k: 2,count: 1,i:0
pca(m): 2,k: 2,count: 6,i:10
pca(m): 2,k: 2,count: 9,i:20
pca(m): 2,k: 2,count: 14,i:30
pca(m): 2,k: 2,count: 19,i:40
pca(m): 2,k: 2,count: 27,i:50
pca(m): 2,k: 2,count: 31,i:60
pca(m): 2,k: 2,count: 35,i:70
pca(m): 2,k: 2,count: 40,i:80
pca(m): 2,k: 2,count: 43,i:90
pca(m): 2,k: 2,count: 48,i:100
pca(m): 2,k: 2,count: 55,i:110
pca(m): 2,k: 2,count: 59,i:120
pca(m): 2,k: 2,count: 62,i:130
pca(m): 2,k: 2,count: 67,i:140
pca(m): 2,k: 2,count: 72,i:150
pca(m): 2,k: 2,count: 79,i:160
pca(m): 2,k: 2,count: 85,i:170
pca(m): 2,k: 2,count: 89,i:180
pca(m): 2,k: 2,count: 95,i:190
pca(m): 2,k: 2,count: 98,i:200
pca(m): 2,k: 2,count: 106,i:210
pca(m): 2,k: 2,count: 111,i:220
pca(m): 2,k: 2,count: 118,i:230
pca(m): 2,k: 2,count: 121,i:240
pca(m): 2,k: 2,count: 125,i:250
pca(m): 2,k: 2,count: 132,i:260
pca(m): 2,k: 2,count: 140,i:270
pca(m): 2,k: 2,count: 145,i:280
pca(m): 2,k: 2,count: 151,i:290
pca(m): 2,k: 2,count: 158,i:300
pca(m): 2,k: 2,count: 162,i:310
pca(m): 2,k

pca(m): 2,k: 1000,count: 193,i:460
pca(m): 2,k: 1000,count: 198,i:470
pca(m): 2,k: 1000,count: 205,i:480
pca(m): 2,k: 1000,count: 209,i:490
pca(m): 2,k: 1000,count: 213,i:500
pca(m): 2,k: 1000,count: 216,i:510
pca(m): 2,k: 1000,count: 222,i:520
pca(m): 2,k: 1000,count: 228,i:530
pca(m): 2,k: 1000,count: 234,i:540
pca(m): 2,k: 1000,count: 239,i:550
pca(m): 2,k: 1000,count: 244,i:560
pca(m): 2,k: 1000,count: 250,i:570
pca(m): 2,k: 1000,count: 254,i:580
pca(m): 2,k: 1000,count: 258,i:590
pca(m): 2,k: 1000,count: 266,i:600
pca(m): 2,k: 1000,count: 269,i:610
pca(m): 2,k: 1000,count: 271,i:620
pca(m): 2,k: 1000,count: 276,i:630
pca(m): 2,k: 1000,count: 284,i:640
pca(m): 2,k: 1000,count: 286,i:650
pca(m): 2,k: 1000,count: 290,i:660
pca(m): 2,k: 1000,count: 298,i:670
pca(m): 2,k: 1000,count: 303,i:680
pca(m): 2,k: 1000,count: 311,i:690
pca(m): 2,k: 1000,count: 316,i:700
pca(m): 2,k: 1000,count: 317,i:710
pca(m): 2,k: 1000,count: 320,i:720
pca(m): 2,k: 1000,count: 323,i:730
pca(m): 2,k: 1000,co

KeyboardInterrupt: 